In [ ]:
#!/usr/bin/env python3
#@author narumeena
#@Description applying random forest on mapped variants of catagories pathogenic and human drive
import numpy as np
import pandas as pd
import dask.dataframe as dd


In [57]:
#pathogenic varaints 

#case = pd.read_csv("../analysis/trainingDataSet/slected_pathogenic_only_SNV.csv",sep=",")
case = pd.read_csv("../analysis/trainingDataSet/slected_pathogenic.csv",sep="\t")
case.head()

,Ref,Alt,Consequence,GC,CpG,motifECount,motifEHIPos,oAA,nAA,cDNApos,...,phyloP100way_vertebrate,SiPhy_29way_logOdds,PROVEAN_score,MetaSVM_score,MetaLR_score,integrated_fitCons_score,phyloP20way_mammalian,phastCons100way_vertebrate,phastCons20way_mammalian,tfbs
0,A,T,INTRONIC,0.384,0.027,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G,C,NON_SYNONYMOUS,0.570,0.080,NaN,NaN,A,P,738.0,...,1.470,8.506,-1.96,0.118,0.647,NaN,0.143,0.952,0.036,NaN
2,G,A,NON_SYNONYMOUS,0.563,0.107,NaN,NaN,G,S,807.0,...,9.869,11.015,-5.81,1.133,0.950,NaN,1.048,1.000,0.942,NaN
3,A,G,NON_SYNONYMOUS,0.576,0.067,NaN,NaN,K,E,625.0,...,1.127,12.576,-0.71,-0.925,0.101,NaN,1.199,0.002,1.000,NaN
4,G,T,NON_SYNONYMOUS,0.642,0.093,NaN,NaN,G,V,962.0,...,7.546,16.019,-5.73,-0.668,0.195,NaN,0.953,1.000,0.914,NaN


In [58]:
len(case)

78176

In [59]:
#number of null values in each column 
pd.options.display.max_rows = 4000
case.isnull().sum()

Ref                               0
Alt                               0
Consequence                       0
GC                                0
CpG                               0
motifECount                   73843
motifEHIPos                   73843
oAA                           41159
nAA                           50156
cDNApos                       39766
relcDNApos                    39766
CDSpos                        41090
relCDSpos                     41090
protPos                       41090
relProtPos                    41090
Domain                        41874
Dst2Splice                    64146
Dst2SplType                   64146
minDistTSS                        0
minDistTSE                        0
SIFTcat                       50649
SIFTval                       50649
PolyPhenCat                   50570
PolyPhenVal                   50570
priPhCons                         0
mamPhCons                         0
verPhCons                         0
priPhyloP                   

In [60]:
#replace null value with 0 
case = case.fillna(0)

In [61]:
pd.options.display.max_rows = 4000
case.isnull().sum()

Ref                           0
Alt                           0
Consequence                   0
GC                            0
CpG                           0
motifECount                   0
motifEHIPos                   0
oAA                           0
nAA                           0
cDNApos                       0
relcDNApos                    0
CDSpos                        0
relCDSpos                     0
protPos                       0
relProtPos                    0
Domain                        0
Dst2Splice                    0
Dst2SplType                   0
minDistTSS                    0
minDistTSE                    0
SIFTcat                       0
SIFTval                       0
PolyPhenCat                   0
PolyPhenVal                   0
priPhCons                     0
mamPhCons                     0
verPhCons                     0
priPhyloP                     0
mamPhyloP                     0
verPhyloP                     0
bStatistic                    0
cHmm_E1 

In [62]:
case.describe()

,GC,CpG,motifECount,motifEHIPos,cDNApos,relcDNApos,CDSpos,relCDSpos,protPos,relProtPos,...,phyloP100way_vertebrate,SiPhy_29way_logOdds,PROVEAN_score,MetaSVM_score,MetaLR_score,integrated_fitCons_score,phyloP20way_mammalian,phastCons100way_vertebrate,phastCons20way_mammalian,tfbs
count,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,...,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000,78176.000000
mean,0.535927,0.070408,0.138713,0.016693,955.806577,0.187102,810.489856,0.235808,270.376228,0.236660,...,5.494295,13.684704,-3.177866,0.385207,0.507064,0.525618,0.812617,0.876181,0.817052,0.000448
std,0.108550,0.063422,0.830378,0.128120,4021.993967,0.244587,3985.643482,0.312713,1328.597068,0.313646,...,3.188045,4.933199,3.198847,0.586056,0.420612,0.268005,0.459457,0.313773,0.329821,0.027930
min,0.212000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-10.473000,0.000000,-13.990000,-1.212000,0.000000,0.000000,-8.346000,0.000000,0.000000,0.000000
25%,0.450000,0.027000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.776000,11.158000,-5.570000,0.000000,0.000000,0.442000,0.852000,1.000000,0.854000,0.000000
50%,0.550000,0.053000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.406000,14.645000,-2.800000,0.252000,0.621000,0.672000,0.951000,1.000000,0.992000,0.000000
75%,0.616000,0.093000,0.000000,0.000000,1104.000000,0.342000,815.000000,0.473000,272.000000,0.474000,...,7.869000,17.317000,0.000000,0.982000,0.936000,0.707000,1.048000,1.000000,0.999000,0.000000
max,0.861000,0.387000,21.000000,1.000000,108115.000000,1.000000,107890.000000,1.000000,35964.000000,1.010000,...,9.869000,28.852000,10.890000,2.337000,1.000000,0.840000,1.199000,1.000000,1.000000,3.000000


In [63]:
#set categories to 1 
case['category']=1


In [64]:
#human drive variants 

control = pd.read_csv("../analysis/trainingDataSet/slected_human_Drive.csv",sep="\t")
control.head()

,Ref,Alt,Consequence,GC,CpG,motifECount,motifEHIPos,oAA,nAA,cDNApos,...,phyloP100way_vertebrate,SiPhy_29way_logOdds,PROVEAN_score,MetaSVM_score,MetaLR_score,integrated_fitCons_score,phyloP20way_mammalian,phastCons100way_vertebrate,phastCons20way_mammalian,tfbs
0,G,A,INTRONIC,0.503,0.040,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,C,INTRONIC,0.530,0.013,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T,C,INTRONIC,0.371,0.040,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,G,INTRONIC,0.391,0.013,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T,C,DOWNSTREAM,0.311,0.013,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
len(control)

1674865

In [66]:
pd.options.display.max_rows = 4000
control.isnull().sum()

Ref                                 0
Alt                                 0
Consequence                         0
GC                                  0
CpG                                 0
motifECount                   1627339
motifEHIPos                   1627339
oAA                           1650987
nAA                           1651002
cDNApos                       1615604
relcDNApos                    1615604
CDSpos                        1650971
relCDSpos                     1650971
protPos                       1650971
relProtPos                    1650971
Domain                        1652008
Dst2Splice                    1660056
Dst2SplType                   1660056
minDistTSS                          0
minDistTSE                          0
SIFTcat                       1666847
SIFTval                       1666847
PolyPhenCat                   1666722
PolyPhenVal                   1666722
priPhCons                        3200
mamPhCons                        3200
verPhCons   

In [67]:
control = control.fillna(0)

In [68]:
pd.options.display.max_rows = 4000
control.isnull().sum()

Ref                           0
Alt                           0
Consequence                   0
GC                            0
CpG                           0
motifECount                   0
motifEHIPos                   0
oAA                           0
nAA                           0
cDNApos                       0
relcDNApos                    0
CDSpos                        0
relCDSpos                     0
protPos                       0
relProtPos                    0
Domain                        0
Dst2Splice                    0
Dst2SplType                   0
minDistTSS                    0
minDistTSE                    0
SIFTcat                       0
SIFTval                       0
PolyPhenCat                   0
PolyPhenVal                   0
priPhCons                     0
mamPhCons                     0
verPhCons                     0
priPhyloP                     0
mamPhyloP                     0
verPhyloP                     0
bStatistic                    0
cHmm_E1 

In [69]:
control.describe()

,GC,CpG,motifECount,motifEHIPos,cDNApos,relcDNApos,CDSpos,relCDSpos,protPos,relProtPos,...,phyloP100way_vertebrate,SiPhy_29way_logOdds,PROVEAN_score,MetaSVM_score,MetaLR_score,integrated_fitCons_score,phyloP20way_mammalian,phastCons100way_vertebrate,phastCons20way_mammalian,tfbs
count,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,...,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06,1.674865e+06
mean,4.482702e-01,3.325465e-02,6.880793e-02,8.849668e-03,6.394288e+01,1.457718e-02,2.530632e+01,7.074250e-03,8.437985e+00,7.093193e-03,...,3.031458e-03,8.160984e-02,6.211223e-03,-9.629533e-03,8.173190e-04,6.688776e-03,-6.882708e-03,3.618044e-03,3.868845e-03,2.089721e-04
std,1.097346e-01,4.375177e-02,5.707814e-01,9.365552e-02,9.829018e+02,9.271886e-02,7.178738e+02,6.853245e-02,2.392976e+02,6.870087e-02,...,2.355213e-01,8.514163e-01,1.584720e-01,9.638825e-02,1.537929e-02,6.548555e-02,1.208955e-01,5.739816e-02,5.717645e-02,2.026662e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-2.000000e+01,0.000000e+00,-1.400000e+01,-1.165000e+00,0.000000e+00,0.000000e+00,-1.012000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.640000e-01,1.300000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.440000e-01,2.700000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.230000e-01,4.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.540000e-01,4.930000e-01,3.000000e+01,1.000000e+00,1.880960e+05,1.000000e+00,1.076210e+05,1.000000e+00,3.587400e+04,1.008000e+00,...,8.034000e+00,2.885200e+01,1.042000e+01,1.761000e+00,9.550000e-01,8.400000e-01,1.045000e+00,1.000000e+00,1.000000e+00,6.000000e+00


In [70]:
#set categories to 0
control['category']=0

In [71]:
#concate case and control dataset 
features = pd.concat([case,control],axis=0, join='outer',ignore_index=True)

In [72]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)

In [73]:
# Display the first 5 rows o
features.head(5)

,GC,CpG,motifECount,motifEHIPos,cDNApos,relcDNApos,CDSpos,relCDSpos,protPos,relProtPos,...,Dst2SplType_ACCEPTOR,Dst2SplType_DONOR,SIFTcat_0,SIFTcat_deleterious,SIFTcat_tolerated,PolyPhenCat_0,PolyPhenCat_benign,PolyPhenCat_possibly_damaging,PolyPhenCat_probably_damaging,PolyPhenCat_unknown
0,0.384,0.027,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,...,0,0,1,0,0,1,0,0,0,0
1,0.570,0.080,0.0,0.0,738.0,0.383,544.0,0.485,182.0,0.488,...,1,0,0,0,1,0,1,0,0,0
2,0.563,0.107,0.0,0.0,807.0,0.418,613.0,0.546,205.0,0.550,...,0,0,0,1,0,0,0,0,1,0
3,0.576,0.067,0.0,0.0,625.0,0.260,439.0,0.397,147.0,0.399,...,0,0,0,0,1,0,0,1,0,0
4,0.642,0.093,0.0,0.0,962.0,0.400,776.0,0.701,259.0,0.704,...,1,0,0,0,1,0,0,0,1,0


In [74]:
features[['category']].head(5)

,category
0,1
1,1
2,1
3,1
4,1


In [75]:
# Labels are the values we want to predict
labels = np.array(features['category'])

In [76]:
# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('category', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [77]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 100)

In [78]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1314780, 168)
Training Labels Shape: (1314780,)
Testing Features Shape: (438261, 168)
Testing Labels Shape: (438261,)


In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 100)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, random_state=42)
# Extract the two most important features
important_indices = [feature_list.index('temp_1'), feature_list.index('average')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]
# Train the random forest
rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');